In [1]:
import os
import scipy
import numpy as np
import pylab as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from pandas import read_csv
import matplotlib.pyplot as plt
from matplotlib import pyplot as mp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.io import savemat, loadmat
from scipy.stats import pointbiserialr
from matplotlib.patches import Circle
from sklearn import datasets, linear_model, preprocessing
from scipy.stats import probplot, pearsonr
from matplotlib.cbook import get_sample_data
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import scale 
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,AnnotationBbox)
# print(__doc__)

/Users/emilyhahn/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Configure environment

In [2]:
dACC_group=[]
L_dlPFC_group=[]
R_dlPFC_group=[]
L_IFG_group=[]
R_IFG_group=[]
conflict_group=[]
adapt_group=[]
rt_group=[]

dACC_group_hc=[]
L_dlPFC_group_hc=[]
R_dlPFC_group_hc=[]
L_IFG_group_hc=[]
R_IFG_group_hc=[]
conflict_group_hc=[]
adapt_group_hc=[]
rt_group_hc=[]

dACC_group_pts=[]
L_dlPFC_group_pts=[]
R_dlPFC_group_pts=[]
L_IFG_group_pts=[]
R_IFG_group_pts=[]
conflict_group_pts=[]
adapt_group_pts=[]
rt_group_pts=[]

regions=['dACC','L_dlPFC','R_dlPFC','L_IFG','R_IFG']
variables=['rt','adapt','conflict']
raw_behav_dir='/Users/emilyhahn/projects/msit_modeling/behavior_preproc/msit_mri_behav'
preproc_behav_dir='/Users/emilyhahn/projects/msit_modeling/behavior_preproc/completed'
LSS_dir='/Users/emilyhahn/projects/msit_modeling/LSS_AVG'
LSS_estim_dir='/Users/emilyhahn/projects/msit_modeling/LSS_AVG_estim'
censor_dir='/Users/emilyhahn/projects/msit_modeling/censor_data'

df=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_I=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_C=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_I_C=pd.DataFrame({"group":[],"subject":[],'rt':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'acc':[],'conflict':[],'adapt':[]})
## Create DFs for:
# df_I_epoch_first45
# df_C_epoch_first45
# df_pts_I_epoch_first45
# df_hcs_I_epoch_first45
# df_pts_C_epoch_first45
# df_hcs_C_epoch_first45

### Define Subjects

In [3]:
SUBJ_LIST = ['hc001','hc002','hc003','hc005','hc006','hc010','hc011',\
             'hc012','hc014','hc015','hc017','hc019','hc021','hc028',\
             'hc031','hc032','hc033','hc034','hc036','hc038','hc042',\
             'pp001','pp002','pp003','pp004','pp005','pp006','pp010',\
             'pp011','pp012','pp013','pp015','pp016']

# SUBJ_LIST = ['hc001','pp001']

### Define Functions - Data Structs

In [4]:
def generate_roi_vars(roi_file,region,var):
    with open(roi_file, 'r') as file:
        for line in file:
            line=line.strip()
            line=float(line)
            if -4<line<4 and line!=0:
#             if -.5<line<.15 and line!=0:
                region.append(line)
            else:
                region.append('NaN')
                print "%s beta exlcuded for %s" % (line,roi_file)
    return
def censor_tps(censor_file,censor_var,var):
    with open(censor_file, 'r') as file:
        for line in file:
            line=line.strip()
            censor_var.append(line)
    return

# def plot_OLS(df_type,variable,group_cond):
#     sns.set()
#     g = sns.lmplot('%s' % variable,'dACC',data=df_type,hue='cond')
#     new_labels = ['C', 'I']
#     for t, l in zip(g._legend.texts, new_labels): t.set_text(l)

#     g = sns.JointGrid('%s' % variable,'dACC',data=df_type,ratio=1000)
#     g.plot_joint(sns.regplot)
#     g.plot(sns.regplot, sns.distplot)
#     g.annotate(stats.pearsonr)
#     g.ax_marg_x.set_axis_off()
#     g.ax_marg_y.set_axis_off()
#     slope, intercept, r_value, p_value, std_err = \
#     stats.linregress(df_type['%s' % variable],df_type['dACC'])
#     plt.title('%s | r val: %.3f | pval: %.3f' % (group_cond,r_value,p_value))
#     plt.show()
#     return

from scipy import stats
def plot_OLS(df_type,variable,group_cond):
    sns.set()
    g=[]
    g = sns.JointGrid(df_type['%s' % variable],df_type['dACC'],ratio=1000)
# #     g = sns.JointGrid(df_type['%s' % variable],df_type['dACC'],ratio=1000)
    g.plot_joint(sns.regplot)
    g.annotate(stats.pearsonr)
#     g.ax_marg_x.set_axis_off()
#     g.ax_marg_y.set_axis_off()
    plt.title('%s' % group_cond)
    plt.show()
    return

### Assemble Indiv and Group Data - SS output, ROI data
    This module generates DataFrames:
    df = ALL subjects, ALL trials
    df_I = ALL subjects, Incongruent trials
    df_C = ALL subjects, Congruent trials
    df_I_C =ALL subjects, Incongruent-Congruent trials
    df_hcs = CTRL subjects, ALL trials
    df_pts = PSYCH subjects, ALL trials
    df_hcs_I = CTRL subjects, Incongruent trials
    df_pts_I = PSYCH subjects, Incongruent trials
    df_hcs_C = CTRL subjects, Congruent trials
    df_pts_C = PSYCH subjects, Congruent trials

In [12]:
for SUBJ in SUBJ_LIST:
    rt=[]
    conflict=[]
    adapt=[]
    dACC=[]
    L_IFG=[]
    R_IFG=[]
    L_dlPFC=[]
    R_dlPFC=[]
    cond=[]
    trial=[]
    acc=[]
    censor=[]        
    #---------------------------------##
    ## Configure SS variables
    #---------------------------------##
    mat = loadmat(os.path.join(preproc_behav_dir,'%s_msit_ss_iter250.mat') % SUBJ)
    #---------------------------------##
    ss_outputs_xsmt = np.expand_dims(np.array([np.concatenate(arr) \
                                for arr in mat['XSmt']]).squeeze(),1)
    ss_xsmt = ss_outputs_xsmt.squeeze()
    rt.extend([float(i) for i in np.array(mat['RT'].squeeze())])
    #---------------------------------##
    conflict.extend([float(i[0]) for i in ss_xsmt])
    adapt.extend([float(i[1]) for i in ss_xsmt])
    acc.extend([float(i) for i in np.array(mat['Accuracy'].squeeze())])
    trial.extend([float(i) for i in np.array(mat['Trial'].squeeze())])
    cond.extend([float(i) for i in np.array(mat['Interference'].squeeze())])
    #---------------------------------##
    ## Configure ROI variables
    #---------------------------------##
    ## Load indiv ROI vars
    file_1=os.path.join(LSS_dir,'%s.dACC_LSS_avg_file.1D' % SUBJ)
    file_2=os.path.join(LSS_dir,'%s.L_IFG_LSS_avg_file.1D' % SUBJ)
    file_3=os.path.join(LSS_dir,'%s.R_IFG_LSS_avg_file.1D' % SUBJ)
    file_4=os.path.join(LSS_dir,'%s.L_dlPFC_LSS_avg_file.1D' % SUBJ)
    file_5=os.path.join(LSS_dir,'%s.R_dlPFC_LSS_avg_file.1D' % SUBJ)
    file_6=os.path.join(censor_dir,'msit.%s.msit_bsm.censor.1D' % SUBJ)
    #---------------------------------##
    ## Generate ROI variables
    generate_roi_vars(file_1,dACC,"dACC")
    generate_roi_vars(file_2,L_IFG,"L_IFG")
    generate_roi_vars(file_3,R_IFG,"R_IFG")
    generate_roi_vars(file_4,L_dlPFC,"L_dlPFC")
    generate_roi_vars(file_5,R_dlPFC,"R_dlPFC")
    censor_tps(file_6,censor,"censor")
    #---------------------------------##
    ## Determine group
    s=[]
    if 'hc' in SUBJ:
        s.append(1)
    elif 'pp' in SUBJ:
        s.append(2)
    GROUP_ARR=np.array(["%s" % s] * len(trial))
    group=[i.strip('[]') for i in GROUP_ARR]
    SUBJ_ARR=np.array(["%s" % SUBJ] * len(trial))
    #---------------------------------## 
    ## Scale + interpolate missing ROI + RT data points
    ## Subjects with more than two points to interpolate excluded

#     dACC = preprocessing.scale(pd.Series(dACC).astype(float).interpolate())
#     R_IFG = preprocessing.scale(pd.Series(R_IFG).astype(float).interpolate())
#     L_IFG = preprocessing.scale(pd.Series(L_IFG).astype(float).interpolate())
#     L_dlPFC = preprocessing.scale(pd.Series(L_dlPFC).astype(float).interpolate())
#     R_dlPFC = preprocessing.scale(pd.Series(R_dlPFC).astype(float).interpolate())
    
    dACC = pd.Series(dACC).astype(float).interpolate()
    R_IFG = pd.Series(R_IFG).astype(float).interpolate()
    L_IFG = pd.Series(L_IFG).astype(float).interpolate()
    L_dlPFC = pd.Series(L_dlPFC).astype(float).interpolate()
    R_dlPFC = pd.Series(R_dlPFC).astype(float).interpolate()
    
    rt = preprocessing.scale(pd.Series(rt).astype(float).interpolate())
    #---------------------------------## 
    dACC_group.append(dACC)
    L_dlPFC_group.append(L_dlPFC)
    R_dlPFC_group.append(R_dlPFC)
    L_IFG_group.append(L_IFG)
    R_IFG_group.append(R_IFG)
    conflict_group.append(conflict)
    adapt_group.append(adapt)
    rt_group.append(rt)
    #---------------------------------##
    if 'pp' in SUBJ_ARR[0]:
        dACC_group_pts.append(dACC)
        L_dlPFC_group_pts.append(L_dlPFC)
        R_dlPFC_group_pts.append(R_dlPFC)
        L_IFG_group_pts.append(L_IFG)
        R_IFG_group_pts.append(R_IFG)
        conflict_group_pts.append(conflict)
        adapt_group_pts.append(adapt)
        rt_group_pts.append(rt)
    elif 'hc' in SUBJ_ARR[0]:
        dACC_group_hc.append(dACC)
        L_dlPFC_group_hc.append(L_dlPFC)
        R_dlPFC_group_hc.append(R_dlPFC)
        L_IFG_group_hc.append(L_IFG)
        R_IFG_group_hc.append(R_IFG)
        conflict_group_hc.append(conflict)
        adapt_group_hc.append(adapt)  
        rt_group_hc.append(rt)
    else:
        "REVIEW SUBJ ID: %s" % SUBJ
    ##---------------------------------##
    ## Append subj data to master DF
    ##---------------------------------##
    df1=pd.DataFrame({"group":group,"subject":SUBJ_ARR,'rt':rt,'dACC':dACC,\
                      'L_IFG':L_IFG,'R_IFG':R_IFG,'L_dlPFC':L_dlPFC,\
                      'R_dlPFC':R_dlPFC,'trial':trial,'cond':cond,'acc':acc,\
                      'conflict':conflict,'adapt':adapt})
    ##---------------------------------##
    df=df.append(df1)
    df1_C=df1[df1.cond == 0]
    df1_I=df1[df1.cond == 1]
    ##---------------------------------##
    ##---------------------------------##
#     plot_OLS(df1,'rt','%s' % SUBJ)
#     plot_OLS(df1,'adapt','%s' % SUBJ)
#     plot_OLS(df1,'conflict','%s' % SUBJ)
    
#     plot_OLS(df1_I,'rt','%s | Incongruent' % SUBJ)
#     plot_OLS(df1_I,'conflict','%s | Incongruent' % SUBJ)
#     plot_OLS(df1_I,'adapt','%s | Incongruent' % SUBJ)
    
#     plot_OLS(df1_C,'rt','%s | Congruent' % SUBJ)
#     plot_OLS(df1_C,'conflict','%s | Congruent' % SUBJ)
#     plot_OLS(df1_C,'adapt','%s | Congruent' % SUBJ)
    ##---------------------------------##
    ##---------------------------------##
    ##---------------------------------##
    
    df1_C_epoch_first45=df1_C[:][0:45]
    df1_C_epoch_first45=df1_C_45.reset_index(drop=True)
    display(df1_C_epoch_first45)
    
    df1_I_epoch_first45=df1_I[:][0:45]
    df1_I_epoch_first45=df1_I_45.reset_index(drop=True)
    display(df1_I_epoch_first45)
    ##---------------------------------##
    df1_I_epoch_first45=df1_I_epoch_first45.mean(axis=0,numeric_only=True)
    df1_C_epoch_first45=df1_C_epoch_first45.mean(axis=0,numeric_only=True)
    
    df1_C=df1_C.mean(axis=0,numeric_only=True)
    df1_I=df1_I.mean(axis=0,numeric_only=True)
    ##---------------------------------##
    S=group[0]
    df1_C['subject']=SUBJ
    df1_C['group']=S
    df1_I['subject']=SUBJ
    df1_I['group']=S
    df1_I_epoch_first45['subject']=SUBJ
    df1_I_epoch_first45['group']=S
    df1_C_epoch_first45['subject']=SUBJ
    df1_C_epoch_first45['group']=S
    ##---------------------------------##
    df_I=df_I.append(df1_I,ignore_index=True)
    df_C=df_C.append(df1_C,ignore_index=True)
    
    df_I_epoch_first45=df_I_epoch_first45.append(df1_I_epoch_first45,ignore_index=True)
    df_C_epoch_first45=df_C_epoch_first45.append(df1_C_epoch_first45,ignore_index=True)
    ##---------------------------------##
    df1_I_C=pd.DataFrame(data={"group":S,"subject":SUBJ,'rt':df1_I['rt']-df1_C['rt']},index=[1])
    df1_I_C={"group":S,"subject":SUBJ,\
             'rt':df1_I['rt']-df1_C['rt'],\
             'dACC':df1_I['dACC']-df1_C['dACC'],\
             'L_IFG': df1_I['L_IFG']-df1_C['L_IFG'],\
             'R_IFG': df1_I['R_IFG']-df1_C['R_IFG'],\
             'L_dlPFC': df1_I['L_dlPFC']-df1_C['L_dlPFC'],\
             'R_dlPFC': df1_I['R_dlPFC']-df1_C['R_dlPFC'],\
             'acc': df1_I['acc']-df1_C['acc'],\
             'conflict': df1_I['conflict']-df1_C['conflict'],\
             'adapt': df1_I['adapt']-df1_C['adapt'] }
    df_I_C=df_I_C.append(df1_I_C,ignore_index=True)
#---------------------------------##
# Substitute Group Identifiers
#---------------------------------##
df=df.replace({'group': '1'}, 0)
df=df.replace({'group': '2'}, 1)

df_I_C=df_I_C.replace({'group': '1'}, 0)
df_I_C=df_I_C.replace({'group': '2'}, 1)

df_I=df_I.replace({'group': '1'}, 0)
df_I=df_I.replace({'group': '2'}, 1)

df_C=df_C.replace({'group': '1'}, 0)
df_C=df_C.replace({'group': '2'}, 1)

df_I_epoch_first45=df_I_epoch_first45.replace({'group': '1'}, 0)
df_I_epoch_first45=df_I_epoch_first45.replace({'group': '2'}, 1)

df_C_epoch_first45=df_C_epoch_first45.replace({'group': '1'}, 0)
df_C_epoch_first45=df_C_epoch_first45.replace({'group': '2'}, 1)
#---------------------------------##
# Remove Trial label
#---------------------------------##
df_C=df_C.drop(labels=['trial'],axis=1)
df_I=df_I.drop(labels=['trial'],axis=1)
df_I_epoch_first45=df_I_epoch_first45.drop(labels=['trial'],axis=1)
df_C_epoch_first45=df_C_epoch_first45.drop(labels=['trial'],axis=1)
# ##---------------------------------##
# ## Parse DataFrames
# ##---------------------------------##
df_hcs=df.where(df['group']==0)
df_hcs=df_hcs.dropna()
df_pts=df.where(df['group']==1)
df_pts=df_pts.dropna()

df_hcs_I=df_I.where(df_I['group']==0)
df_hcs_I=df_hcs_I.dropna()
df_pts_I=df_I.where(df_I['group']==1)
df_pts_I=df_pts_I.dropna()

df_hcs_C=df_C.where(df_C['group']==0)
df_hcs_C=df_hcs_C.dropna()
df_pts_C=df_C.where(df_C['group']==1)
df_pts_C=df_pts_C.dropna()

df_hcs_I_epoch_first45=df_hcs_I_epoch_first45.where(df_hcs_I_epoch_first45['group']==0)
df_hcs_I_epoch_first45=df_hcs_I_epoch_first45.dropna()
df_hcs_C_epoch_first45=df_hcs_C_epoch_first45.where(df_hcs_C_epoch_first45['group']==0)
df_hcs_C_epoch_first45=df_hcs_C_epoch_first45.dropna()

df_pts_I_epoch_first45=df_pts_I_epoch_first45.where(df_pts_I_epoch_first45['group']==1)
df_pts_I_epoch_first45=df_pts_I_epoch_first45.dropna()
df_pts_C_epoch_first45=df_pts_C_epoch_first45.where(df_pts_C_epoch_first45['group']==1)
df_pts_C_epoch_first45=df_pts_C_epoch_first45.dropna()
##---------------------------------##
# df_pts_I=df_pts_I.reset_index(drop=True)
# display(df_pts_I)

# df_hcs_I=df_hcs_I.reset_index(drop=True)
# display(df_hcs_I)

# df_pts_C=df_pts_C.reset_index(drop=True)
# display(df_pts_C)

# df_hcs_C=df_hcs_C.reset_index(drop=True)
# display(df_hcs_C)
##---------------------------------##
# ## PTS INCONG FIG
# slope,intercept,r_value,p_value,std_err=stats.linregress(df_pts_I['rt'], df_pts_I['dACC'])
# print 'rval: %s | pval: %s' % (r_value,p_value)
# sns.set()
# sns.jointplot(x='rt', y='dACC', data=df_pts_I,kind="reg",stat_func=stats.pearsonr)
# plt.show()

## HCS INCONG FIG
# slope,intercept,r_value,p_value,std_err=stats.linregress(df_hcs_I['rt'], df_hcs_I['dACC'])
# print 'rval: %s | pval: %s' % (r_value,p_value)
# sns.set()
# sns.jointplot(x='rt', y='dACC', data=df_hcs_I,kind="reg",stat_func=stats.pearsonr)
# plt.show()

## PTS CONG FIG
# slope,intercept,r_value,p_value,std_err=stats.linregress(df_pts_C['rt'], df_pts_C['dACC'])
# print 'rval: %s | pval: %s' % (r_value,p_value)
# sns.set()
# sns.jointplot(x='rt', y='dACC', data=df_pts_C,kind="reg")
# plt.show()
# sns.jointplot(x='rt', y='dACC', data=df_pts_C,kind="reg",stat_func=stats.pearsonr)
# plt.show()

## HCS CONG FIG
# slope,intercept,r_value,p_value,std_err=stats.linregress(df_hcs_C['rt'], df_hcs_C['dACC'])
# print 'rval: %s | pval: %s' % (r_value,p_value)
# sns.set()
# # sns.jointplot(x='rt', y='dACC', data=df_hcs_C,kind="reg")
# # plt.show()
# sns.jointplot(x='rt', y='dACC', data=df_hcs_C,kind="reg",stat_func=stats.pearsonr)
# plt.show()


,L_IFG,L_dlPFC,R_IFG,R_dlPFC,acc,adapt,cond,conflict,dACC,group,rt,subject,trial
0,-0.071589,0.317506,1.857820,0.571066,1.0,-1.171227,0.0,0.456042,0.457386,1,-0.553247,hc001,1.0
1,-0.098958,0.578367,1.446660,0.349529,1.0,-1.171593,0.0,0.454676,0.233562,1,-0.698518,hc001,3.0
2,0.018505,0.482988,0.958111,0.482485,1.0,-1.171886,0.0,0.453432,-0.052651,1,-0.745646,hc001,4.0
3,0.290993,0.330457,0.549144,0.534948,1.0,-1.172180,0.0,0.452189,-0.150036,1,-0.985133,hc001,5.0
4,1.193400,-0.208444,0.467648,-0.092879,1.0,-1.172382,0.0,0.450946,-0.043763,1,-0.544931,hc001,6.0
5,-0.229261,0.104435,-0.269049,0.103478,1.0,-1.173444,0.0,0.445807,-0.072098,1,-0.851528,hc001,9.0
6,-0.754711,-0.026994,-0.829545,0.342677,1.0,-1.173750,0.0,0.444164,-0.191109,1,-0.380563,hc001,10.0
7,-0.158323,-0.021684,-1.086060,0.187894,1.0,-1.174815,0.0,0.439729,-0.176305,1,-0.192554,hc001,12.0
8,0.785523,-0.167424,-0.694752,-0.360460,1.0,-1.176918,0.0,0.433611,0.270245,1,-0.544161,hc001,15.0
9,-0.470141,0.089870,0.404199,0.495060,1.0,-1.179337,0.0,0.427050,0.215692,1,-0.511626,hc001,18.0


,L_IFG,L_dlPFC,R_IFG,R_dlPFC,acc,adapt,cond,conflict,dACC,group,rt,subject,trial
0,0.120258,0.477321,0.412547,-0.241151,1.0,-1.051862,0.0,0.190091,1.189460,1,-1.063711,hc002,1.0
1,-0.339825,0.173051,0.424758,-0.281682,1.0,-1.051823,0.0,0.189383,0.144169,1,0.241141,hc002,3.0
2,-0.623503,0.049876,0.289904,-0.323630,1.0,-1.052202,0.0,0.188690,-0.311344,1,-1.185544,hc002,4.0
3,-0.795792,0.064696,0.116487,-0.305464,1.0,-1.052397,0.0,0.187998,-0.584038,1,-0.686759,hc002,5.0
4,-0.669636,0.231571,-0.050449,-0.201743,1.0,-1.052581,0.0,0.187305,-0.552375,1,-1.291789,hc002,6.0
5,-0.807903,-0.241193,-0.091028,-0.305181,1.0,-1.052178,0.0,0.185970,-0.676832,1,-0.401573,hc002,9.0
6,-0.765285,-0.063633,-0.008145,-0.282223,1.0,-1.052064,0.0,0.185681,-0.570506,1,-1.204361,hc002,10.0
7,-0.714024,0.361497,0.230123,-0.138222,1.0,-1.051205,0.0,0.185746,-0.006712,1,-1.146345,hc002,12.0
8,0.748882,-0.300675,-0.200771,0.224273,1.0,-1.049289,0.0,0.186138,0.187103,1,-1.233420,hc002,15.0
9,-0.745472,0.104175,0.167915,0.029110,1.0,-1.046507,0.0,0.187167,0.493261,1,1.424665,hc002,18.0


,L_IFG,L_dlPFC,R_IFG,R_dlPFC,acc,adapt,cond,conflict,dACC,group,rt,subject,trial
0,0.203603,-0.227421,-0.258095,0.330905,1.0,-1.019265,0.0,0.160360,-0.047227,1,0.202442,hc003,1.0
1,0.418896,0.557518,-0.267547,0.668681,1.0,-1.027917,0.0,0.158555,0.138272,1,1.746680,hc003,3.0
2,0.654119,0.950106,-0.412319,0.425164,1.0,-1.042530,0.0,0.156818,0.206009,1,-1.429718,hc003,4.0
3,0.718345,0.935776,-0.362577,0.383476,1.0,-1.051480,0.0,0.155082,0.202531,1,-0.850664,hc003,5.0
4,0.474880,0.321311,0.001683,0.290055,1.0,-1.057943,0.0,0.153346,0.051037,1,-0.724311,hc003,6.0
5,0.870268,0.439282,0.124773,0.309289,1.0,-1.062970,0.0,0.150627,0.175189,1,-0.376682,hc003,9.0
6,0.492937,0.059426,0.079709,0.012551,1.0,-1.062790,0.0,0.150228,0.157187,1,0.491549,hc003,10.0
7,-0.313395,-0.488745,0.535470,-0.309084,1.0,-1.072080,0.0,0.148801,-0.081643,1,-0.106966,hc003,12.0
8,-0.098388,-0.348250,0.283633,-0.339795,1.0,-1.087252,0.0,0.147516,-0.104568,1,-0.479515,hc003,15.0
9,-0.363976,-0.355043,-0.420071,-0.078586,1.0,-1.095586,0.0,0.148296,-0.018209,1,-0.003363,hc003,18.0


8.68087 beta exlcuded for /Users/emilyhahn/projects/msit_modeling/LSS_AVG/hc005.dACC_LSS_avg_file.1D
4.02871 beta exlcuded for /Users/emilyhahn/projects/msit_modeling/LSS_AVG/hc005.R_IFG_LSS_avg_file.1D


,L_IFG,L_dlPFC,R_IFG,R_dlPFC,acc,adapt,cond,conflict,dACC,group,rt,subject,trial
0,-0.071793,0.332398,0.662779,0.035675,1.0,-1.343927,0.0,0.301419,0.184903,1,-0.806099,hc005,1.0
1,0.030848,-0.013891,1.241220,-0.211101,1.0,-1.344395,0.0,0.300318,-0.772910,1,-0.551168,hc005,3.0
2,0.105534,0.209557,1.716010,-0.202614,1.0,-1.345167,0.0,0.299285,-0.702600,1,-0.768236,hc005,4.0
3,0.011582,0.249158,1.948820,-0.160114,1.0,-1.345830,0.0,0.298251,-0.529073,1,-0.628122,hc005,5.0
4,0.205347,-0.347040,0.553504,-0.132690,1.0,-1.346552,0.0,0.297218,-0.152075,1,-1.338021,hc005,6.0
5,0.535284,-0.507822,-0.300120,-0.237089,1.0,-1.344482,0.0,0.296181,0.206268,1,-0.286854,hc005,9.0
6,0.782490,-0.566117,-0.266011,-0.214517,1.0,-1.344057,0.0,0.296041,0.618403,1,-0.080675,hc005,10.0
7,0.284421,-0.448013,0.134442,-0.083166,1.0,-1.344884,0.0,0.295302,-0.214593,1,-0.633260,hc005,12.0
8,-0.251996,0.121243,-0.588871,0.159909,1.0,-1.344863,0.0,0.295910,-0.716172,1,-0.332424,hc005,15.0
9,-0.197936,0.771218,2.365190,-0.024810,1.0,-1.345974,0.0,0.296556,-0.168910,1,-0.448525,hc005,18.0


...

,L_IFG,L_dlPFC,R_IFG,R_dlPFC,acc,adapt,cond,conflict,dACC,group,rt,subject,trial
0,-0.054896,-0.407719,-0.051680,0.222533,1.0,-1.590451,0.0,0.262330,0.145070,2,-0.672639,pp011,1.0
1,-0.321736,-0.258506,0.199328,0.069238,1.0,-1.590970,0.0,0.262258,0.161665,2,-0.246154,pp011,3.0
2,-0.285709,-0.166605,0.269498,-0.169725,1.0,-1.591936,0.0,0.262131,0.034772,2,-1.564281,pp011,4.0
3,-0.182627,-0.250401,0.382810,-0.304036,1.0,-1.592030,0.0,0.262003,-0.091095,2,-1.017678,pp011,5.0
4,0.385978,-0.386311,0.363530,-0.154163,1.0,-1.591940,0.0,0.261876,-0.083505,2,-1.272560,pp011,6.0
5,-0.063670,0.169608,0.000936,0.495046,1.0,-1.589188,0.0,0.262302,0.089078,2,-1.218965,pp011,9.0
6,0.212940,0.229800,0.161365,0.264604,1.0,-1.587293,0.0,0.262877,0.095204,2,-0.996984,pp011,10.0
7,0.843498,0.824065,0.553136,0.152093,1.0,-1.583299,0.0,0.263910,0.144550,2,-0.721574,pp011,12.0
8,1.016570,0.726902,0.086904,0.412268,1.0,-1.577027,0.0,0.265939,0.002032,2,-0.683958,pp011,15.0
9,0.511596,0.105452,0.508720,0.421089,1.0,-1.570237,0.0,0.268677,0.176828,2,-0.638352,pp011,18.0


,L_IFG,L_dlPFC,R_IFG,R_dlPFC,acc,adapt,cond,conflict,dACC,group,rt,subject,trial
0,0.294158,-0.027541,-0.334920,0.075780,1.0,-1.096062,0.0,0.350083,0.102063,2,-0.311268,pp012,1.0
1,0.157244,-0.455416,0.141514,0.402520,1.0,-1.093187,0.0,0.350601,0.402853,2,-0.168343,pp012,3.0
2,0.068700,-0.426176,0.347000,0.183637,1.0,-1.091167,0.0,0.351053,0.400046,2,-0.160961,pp012,4.0
3,0.034246,-0.358208,0.466027,0.007729,1.0,-1.089749,0.0,0.351504,0.300618,2,-0.577346,pp012,5.0
4,0.179810,-0.141310,0.727441,0.363894,1.0,-1.086039,0.0,0.351955,0.184685,2,-0.447156,pp012,6.0
5,-0.027781,0.546051,0.720666,0.576953,1.0,-1.071880,0.0,0.353097,0.135705,2,-0.289965,pp012,9.0
6,-0.027270,0.363694,0.795686,0.331732,1.0,-1.066874,0.0,0.353411,0.245917,2,0.187270,pp012,10.0
7,-0.029584,0.411166,0.708373,-0.263054,1.0,-1.063068,0.0,0.353740,0.522381,2,-0.131856,pp012,12.0
8,-0.082876,-0.214651,0.686418,-0.045955,1.0,-1.056115,0.0,0.354504,0.790481,2,-0.168304,pp012,15.0
9,0.333453,0.106422,0.021662,-0.215255,1.0,-1.042610,0.0,0.356208,0.358751,2,-0.419007,pp012,18.0


,L_IFG,L_dlPFC,R_IFG,R_dlPFC,acc,adapt,cond,conflict,dACC,group,rt,subject,trial
0,0.055668,-0.005340,0.303477,0.032840,1.0,-1.491600,0.0,0.287212,-0.034633,2,-0.909422,pp013,1.0
1,0.090558,0.005671,1.416480,0.192343,1.0,-1.490239,0.0,0.287900,0.062305,2,-0.377935,pp013,3.0
2,0.178740,0.013748,1.861470,0.357565,1.0,-1.489570,0.0,0.288457,0.132220,2,-0.477962,pp013,4.0
3,0.249634,0.014082,1.777960,0.316345,1.0,-1.489119,0.0,0.289014,0.147130,2,-0.686182,pp013,5.0
4,0.413646,-0.004168,0.657627,-0.008281,1.0,-1.488604,0.0,0.289571,0.150856,2,-1.079875,pp013,6.0
5,0.189234,0.018175,-0.492339,-0.134617,1.0,-1.485767,0.0,0.290847,0.003920,2,-0.024316,pp013,9.0
6,0.287861,0.019872,-0.487853,-0.195569,1.0,-1.485716,0.0,0.291185,0.022779,2,0.301756,pp013,10.0
7,0.318549,0.013715,-0.175705,-0.157123,1.0,-1.487537,0.0,0.292069,0.053951,2,-0.650098,pp013,12.0
8,-0.095939,-0.004404,-0.413826,-0.087997,1.0,-1.487786,0.0,0.294921,-0.042408,2,-0.620993,pp013,15.0
9,0.041319,-0.006952,0.659444,0.295143,1.0,-1.488873,0.0,0.297276,0.015479,2,-0.642370,pp013,18.0


,L_IFG,L_dlPFC,R_IFG,R_dlPFC,acc,adapt,cond,conflict,dACC,group,rt,subject,trial
0,-0.516292,-0.051914,-0.223615,0.078455,1.0,-1.410713,0.0,0.252193,-0.076863,2,-0.895328,pp015,1.0
1,-0.624493,-0.062086,-0.378848,-0.030637,1.0,-1.410111,0.0,0.251948,-0.062678,2,-1.316939,pp015,3.0
2,-0.574608,-0.059096,-0.451586,-0.077551,1.0,-1.408854,0.0,0.251831,-0.092447,2,-1.228999,pp015,4.0
3,-0.584687,-0.053318,-0.436996,-0.086053,1.0,-1.406810,0.0,0.251715,-0.102277,2,-1.269808,pp015,5.0
4,-0.614213,-0.043349,-0.500469,-0.126194,1.0,-1.403893,0.0,0.251599,-0.084309,2,-0.941277,pp015,6.0
5,0.249179,-0.023905,0.341538,-0.110889,1.0,-1.395831,0.0,0.250560,-0.150710,2,-0.578567,pp015,9.0
6,0.618106,-0.020689,0.508738,-0.126809,1.0,-1.393134,0.0,0.250306,-0.042333,2,-0.677610,pp015,10.0
7,1.105630,0.003117,0.799522,-0.008817,1.0,-1.386889,0.0,0.250196,0.588525,2,-1.022896,pp015,12.0
8,1.287870,0.008097,0.546935,0.188533,1.0,-1.374305,0.0,0.250696,0.693486,2,0.122748,pp015,15.0
9,0.846963,0.004330,0.653724,-0.068784,1.0,-1.366607,0.0,0.250462,0.007566,2,-0.933876,pp015,18.0


,L_IFG,L_dlPFC,R_IFG,R_dlPFC,acc,adapt,cond,conflict,dACC,group,rt,subject,trial
0,1.546410,-0.320549,0.160829,0.629876,1.0,-1.481281,0.0,0.293252,-0.061939,2,-0.628400,pp016,1.0
1,-0.327852,-0.308112,-0.383629,-0.437558,1.0,-1.480927,0.0,0.297168,-0.066664,2,-0.666566,pp016,3.0
2,-0.652537,-0.296811,-0.465384,-0.708901,1.0,-1.480557,0.0,0.301082,-0.061940,2,-0.977381,pp016,4.0
3,-0.530954,-0.296129,-0.474667,-0.618303,1.0,-1.479867,0.0,0.304996,-0.058998,2,-1.058441,pp016,5.0
4,0.463318,-0.246278,-0.176117,-0.187133,1.0,-1.478765,0.0,0.308910,-0.070495,2,-0.903558,pp016,6.0
5,0.339496,-0.252345,-0.374485,-0.268378,1.0,-1.474793,0.0,0.319820,-0.070879,2,-0.434826,pp016,9.0
6,0.389651,-0.240012,-0.282530,-0.274501,1.0,-1.473588,0.0,0.324258,-0.070422,2,-0.013772,pp016,10.0
7,0.004759,-0.225168,-0.231329,-0.449157,1.0,-1.471913,0.0,0.337076,-0.047217,2,-0.850336,pp016,12.0
8,-0.582712,-0.173014,-0.500197,-0.565695,1.0,-1.468814,0.0,0.355594,0.034107,2,-0.291032,pp016,15.0
9,-0.538470,0.262008,-0.003099,0.897898,1.0,-1.468111,0.0,0.358886,0.010772,2,0.172332,pp016,18.0


In [ ]:
print 'slope: %.2f \nintercept: %.2f \nr value: %.2f \np value: %.2f \nSD: %.2f' % \
(slope,intercept,r_value,p_value,std_err)

### Visualize Main Findings

In [ ]:
# df['conflict_X_group']=df['conflict']*df['group']
# df['adapt_X_group']=df['adapt']*df['group']
# df_I['conflict_X_group']=df_I['conflict']*df_I['group']
# df_C['conflict_X_group']=df_C['conflict']*df_C['group']
# df_I['adapt_X_group']=df_I['adapt']*df_I['group']
# df_C['adapt_X_group']=df_C['adapt']*df_C['group']
# df_I['rt_X_group']=df_I['rt']*df_I['group']
# df_C['rt_X_group']=df_C['rt']*df_C['group']

# plot_OLS(df_pts_I,'rt','PTS | Incongruent')
# plot_OLS(df_hcs_I,'rt','CTRLS | Incongruent')
# plot_OLS(df_pts_C,'rt','PTS | Congruent')
# plot_OLS(df_hcs_C,'rt','CTRLS | Congruent')

# plot_OLS(df_pts_I,'conflict','PTS | Incongruent')
# plot_OLS(df_hcs_I,'conflict','CTRLS | Incongruent')
# plot_OLS(df_pts_C,'conflict','PTS | Congruent')
# plot_OLS(df_hcs_C,'conflict','CTRLS | Congruent')

# plot_OLS(df_pts_I,'adapt','PTS | Incongruent')
# plot_OLS(df_hcs_I,'adapt','CTRLS | Incongruent')
# plot_OLS(df_pts_C,'adapt','PTS | Congruent')
# plot_OLS(df_hcs_C,'adapt','CTRLS | Congruent')

# plot_OLS(df,'conflict_X_group','ALL SUBJS | ALL COND | dACC ~ conflict * group')
# plot_OLS(df_I,'conflict_X_group','ALL SUBJS | Incongruent | dACC ~ conflict * group')
# plot_OLS(df_C,'conflict_X_group','ALL SUBJS | Congruent | dACC ~ conflict * group')

# plot_OLS(df_I,'adapt_X_group','ALL SUBJS | Incongruent | dACC ~ adapt * group')
# plot_OLS(df_C,'adapt_X_group','ALL SUBJS | Congruent | dACC ~ adapt * group')

In [ ]:
# g=sns.lmplot(x='conflict',y='dACC',data=df_I,hue='group')
# plt.show()

# g=sns.lmplot(x='adapt',y='dACC',data=df_I,hue='group')
# plt.show()

# g=sns.lmplot(x='rt',y='dACC',data=df_I,hue='group')
# plt.show()

# g=sns.lmplot(x='conflict',y='dACC',data=df_C,hue='group')
# plt.show()

# g=sns.lmplot(x='adapt',y='dACC',data=df_C,hue='group')
# plt.show()

# g=sns.lmplot(x='rt',y='dACC',data=df_C,hue='group')
# plt.show()

# g=sns.lmplot(x='conflict_X_group',y='dACC',data=df,hue='group',markers=['o','x'])
# new_title = 'Group'
# g._legend.set_title(new_title)
# new_labels = ['HC', 'PT']
# for t, l in zip(g._legend.texts, new_labels): t.set_text(l)
# ## Stats from P3 notebook below
# plt.title("%s | %s" % ('ALL TRIALS','r = 0.5612 | SD = 0.189 \n T = 2.974 | p<0.003'))
# plt.show()

# g=sns.lmplot(x='rt_X_group',y='dACC',data=df_I,hue='group')
# new_title = 'Group'
# g._legend.set_title(new_title)
# new_labels = ['HC', 'PT']
# for t, l in zip(g._legend.texts, new_labels): t.set_text(l)
# ## Stats from P3 notebook below
# plt.title("%s | %s" % ('INCONGRUENT','r = 0.18 | SD = 0.075 \n T = 2.459 | p<0.016'))
# plt.show()

# g=sns.lmplot(x='rt_X_group',y='dACC',data=df_C,hue='group')
# new_title = 'Group'
# g._legend.set_title(new_title)
# new_labels = ['HC', 'PT']
# for t, l in zip(g._legend.texts, new_labels): t.set_text(l)
# ## Stats from P3 notebook below
# plt.title("%s | %s" % ('CONGRUENT','r = 0.2127 | SD = 0.072 \n T = -2.935 | p<0.004'))
# plt.show()

### Plot ROI ~ SS var | Linear Reg w/ Cond Hue

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

# For 3d plots. This import is necessary to have 3D plotting below
from mpl_toolkits.mplot3d import Axes3D

# For statistics. Requires statsmodels 5.0 or more
from statsmodels.formula.api import ols
# Analysis of Variance (ANOVA) on linear models
from statsmodels.stats.anova import anova_lm
def lmplots_ss(df_type,sub_or_group):
    ##---------------------------------##
    ##  Display regplots
    ##---------------------------------##  
#     target_rois=['dACC','L_IFG','R_IFG','L_dlPFC','R_dlPFC']
    target_rois=['dACC']
    def make_plots(ss_var,target_rois):
        for region in target_rois:
            sns.set()
            slope, intercept, r_value, p_value, std_err = \
            stats.linregress(df_type['%s' % ss_var],df_type['%s' % region])
            
            g=sns.lmplot(x="%s" % ss_var,y="%s" % region,data=df_type,hue="group")           
            new_title = 'Group'
            g._legend.set_title(new_title)
            new_labels = ['HC', 'PT']
            for t, l in zip(g._legend.texts, new_labels): t.set_text(l)

#             g=sns.lmplot(y="%s" % ss_var,x="%s" % region,data=df_type)
            
            plt.title("%s r = %.2f | p = %s" % (sub_or_group,r_value,p_value))
            g.set_axis_labels('%s' % ss_var, '%s' % region)
            plt.show()
        return
    make_plots("conflict",target_rois)  
    make_plots("adapt",target_rois)
    make_plots('rt',target_rois)
    return
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot Group Data
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# lmplots_ss(df,'ALL SUBJS')
# lmplots_ss(df_hcs,'CONTROL SUBJS')
# lmplots_ss(df_pts,'PSYCHIATRIC SUBJS')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot AVERAGED Group Data
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# lmplots_ss(df_I_C,'ALL SUBJS | Incongruent-Congruent \n')
# lmplots_ss(df_I,'ALL SUBJS | Incongruent \n')
# lmplots_ss(df_C,'ALL SUBJS | Congruent \n ')
lmplots_ss(df_hcs_I,'CTRL SUBJS | Incongruent \n')
lmplots_ss(df_hcs_C,'CTRL SUBJS | Congruent \n')
lmplots_ss(df_pts_I,'PSYCH SUBJS | Incongruent \n')
lmplots_ss(df_pts_C,'PSYCH SUBJS | Congruent \n')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot Individual Data
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# for subj in SUBJ_LIST:
#     df_sub=df.where(df['subject']==subj)
#     df_sub=df_sub.dropna()
#     lmplots_ss(df_sub,subj.upper())

### Plot ROI ~ ROI | Linear Reg w/ Cond Hue

In [ ]:
def lmplots(df_type,sub_or_group):
    #---------------------------------##
    #  Display regplots
    #---------------------------------##  
    ## Uncomment for "Plot Group Data"
    #  sns.set_style("darkgrid",{"axes.facecolor":".85"})
    #  sns.set_palette(sns.hls_palette(8, l=.3, s=.8))
    #  plt.style.use('fivethirtyeight')
    sns.set()
    target_rois=['L_IFG','R_IFG','L_dlPFC','R_dlPFC']
    def make_plots(roi,target_rois):    
        for region in target_rois:

            g=sns.lmplot(x="%s" % roi,y="%s" % region,data=df_type,hue="group",markers=["o", "x"])
#             g=sns.lmplot(x="%s" % roi,y="%s" % region,data=df_type,hue='cond')
            plt.title("%s | r = %.2f | p = %s" % (sub_or_group,r_value,p_value))
            g.set_axis_labels('%s' % roi, '%s' % region)
            new_title = 'Group'
            g._legend.set_title(new_title)
            new_labels = ['C', 'P']
            for t, l in zip(g._legend.texts, new_labels): t.set_text(l)
            plt.show()
        return
    make_plots("dACC",target_rois)  
    make_plots("L_IFG",target_rois[1:])
    make_plots("R_IFG",target_rois[2:])   
    make_plots("L_dlPFC",target_rois[3:])
    return
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot Group Data
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# lmplots(df,'ALL SUBJS')
# lmplots(df_hcs,'CONTROL SUBJS')
# lmplots(df_pts,'PSYCHIATRIC SUBJS')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot Data by Conditon
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# lmplots(df_I,'INCONGRUENT')
# lmplots(df_C,'CONGRUENT')
# lmplots(df_I_C,'INCONGRUENT-CONGRUENT')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot Individual Data
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# for subj in SUBJ_LIST:
#     df_sub=df.where(df['subject']==subj)
#     df_sub=df_sub.dropna()
#     lmplots(df_sub,subj.upper())

### Plot Joint Distributions

In [ ]:
def jpd_corr_plots(df_type1,df_type2,sub_or_group):
    ##---------------------------------##
    ##  Display JPD + correlations
    ##---------------------------------## 
    target_rois=['L_IFG','R_IFG','L_dlPFC','R_dlPFC']
    def make_joint_plots(roi,target_rois):   
        for region in target_rois:
            sns.set()
            g = sns.kdeplot(df_type1["%s" % roi],df_type1["%s" % region],\
                    cmap="Blues", shade=True, shade_lowest=False, cbar=True, labels='HCS')
            g = sns.kdeplot(df_type2["%s" % roi],df_type2["%s" % region], \
                cmap="Oranges", shade=False, shade_lowest=False, cbar=True, labels='PTS')
            plt.legend()
            plt.title("oranges = patients | blues = controls")
            plt.show()       
        return
    make_joint_plots("dACC",target_rois)  
    make_joint_plots("L_IFG",target_rois[1:])
    make_joint_plots("R_IFG",target_rois[2:])   
    make_joint_plots("L_dlPFC",target_rois[3:])
    return
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot ALL Group Data
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# jpd_corr_plots(df_hcs,df_pts,'ALL SUBJS')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot AVERAGED Group Data
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# jpd_corr_plots(df_hcs_I,df_pts_I,'Incongruent ALL SUBJS')
# jpd_corr_plots(df_hcs_C,df_pts_C,'Congruent ALL SUBJS')

### Joint plots and pair grids

In [ ]:
def pairgrid_plot(df_type,sub_or_group):
    sns.set()
    sns.axes_style('white')
    palette=sns.color_palette("Paired", 2)
    vars=["dACC","L_IFG","R_IFG","L_dlPFC","R_dlPFC","conflict","adapt","rt"]
#     g = sns.PairGrid(df_type,vars=vars,hue='subject')
    g = sns.PairGrid(df_type,vars=vars,hue='cond')
    g.map(sns.regplot)
    g.add_legend()
    plt.title('%s' % sub_or_group)
    plt.show()
    return
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot ALL Data
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# pairgrid_plot(df,'ALL SUBJS')
# pairgrid_plot(df_hcs,'CONTROL SUBJS')
# pairgrid_plot(df_pts,'PSYCHIATRIC SUBJS')
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot AVERAGED Group Data
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# pairgrid_plot(df_I,'Incongruent ALL SUBJS')
# pairgrid_plot(df_hcs_I,'Incongruent CONTROL SUBJS')
# pairgrid_plot(df_pts_I,'Incongruent PSYCHIATRIC SUBJS')
# pairgrid_plot(df_C,'Congruent ALL SUBJS')
# pairgrid_plot(df_hcs_C,'Congruent CONTROL SUBJS')
# pairgrid_plot(df_pts_C,'Congruent PSYCHIATRIC SUBJS')
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot Individual Data
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# for subj in SUBJ_LIST:
#     df_sub=df.where(df['subject']==subj)
#     df_sub=df_sub.dropna()
#     pairgrid_plot(df_sub,subj.upper())

### Distributions of Variables

In [ ]:
def df_dist(regions,df_type,subj_group,behav_vars):
    for region in regions:
        sns.distplot(df_type['%s' % region],label='%s' % region)
    plt.legend()
    plt.title(subj_group)
    plt.xlabel('ALL ROIs')
    plt.show()
    for region in regions:
        sns.distplot(df_type['%s' % region])
        plt.title(subj_group)
        plt.show()
    for var in variables:
        sns.distplot(df_type['%s' % var],label='%s' % var)
#         sns.distplot(df_type['%s' % var], kde=False, fit=stats.gamma)
    plt.legend()
    plt.title(subj_group)
    plt.xlabel('ALL Variables')
    plt.show()
    for var in variables:
        sns.distplot(df_type['%s' % var])
#         sns.distplot(df_type['%s' % var], kde=False, fit=stats.gamma)
        plt.title(subj_group)
        plt.show()
    return
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Plot AVERAGED Group Data
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# df_dist(regions,df_I,'Incongruent ALL SUBJS',variables)
# df_dist(regions,df_hcs_I,'Incongruent CONTROL SUBJS',variables)
# df_dist(regions,df_pts_I,'Incongruent PSYCHIATRIC SUBJS',variables)
# df_dist(regions,df_C,'Congruent ALL SUBJS',variables)
# df_dist(regions,df_hcs_C,'Congruent CONTROL SUBJS',variables)
# df_dist(regions,df_pts_C,'Congruent PSYCHIATRIC SUBJS',variables)

### Linear Mixed Effects Model

In [ ]:
# def mixed_LM(df_type,sub_or_group):
#     for beta in ['dACC','L_IFG','R_IFG','L_dlPFC','R_dlPFC']:
#         for ss_v in ['conflict','adapt','rt']:
#             md = smf.mixedlm('%s ~ %s' % (reg,ss_v),data=df_type,groups="group")
#             mdf = md.fit()
#             print(mdf.summary())
#     return
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# mixed_LM(df,'ALL SUBJS')
# mixed_LM(df_hcs,'CONTROL SUBJS')
# mixed_LM(df_pts,'PSYCHIATRIC SUBJS')
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

### *DEPRECIATED: Modify this section to reflect updated ss model*

The MSIT state space model is as follows:

$$
RT(k) = \beta_0 + \beta_1 * Interference + X_{ss}(k)
$$
where
$$
X_{ss}(k) = X_{ss}(k - 1) + W
$$

Interference is 1 if the current trial k is an incongruent trial and 0 if it is a congruent trial.